In [1]:
import pandas as pd
import numpy as np
import time
import yfinance as yf

In [2]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [3]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]
    
    #time.sleep(0.5)

HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 
HTTP Error 401: 


In [4]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,NaN,NaN,NaN
EFA,iShares MSCI EAFE ETF,ETF,USD,NaN,NaN,NaN
EEM,iShares MSCI Emerging Index Fun,ETF,USD,NaN,NaN,NaN
PSP,Invesco Global Listed Private E,ETF,USD,NaN,NaN,NaN
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,NaN,NaN,NaN
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,NaN,NaN,NaN
DBC,Invesco DB Commodity Index Trac,ETF,USD,NaN,NaN,NaN
IYR,iShares U.S. Real Estate ETF,ETF,USD,6211444,3480166144,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,7506784,34938195968,The underlying index measures the performance ...


In [5]:
STARTDATE = '2011-01-01'
ENDDATE = '2025-05-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_35338/1766415481.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Close']
[*********************100%***********************]  12 of 12 completed


In [6]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()
rets['SHV'] -= .01/20

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_35338/1814017950.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [7]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2011-02-28,0.006617,0.041710,-0.000437,0.035498,0.014353,-0.002084,0.045204,0.040146,0.002583,-0.000410,0.034738,0.007208
2011-03-31,0.008427,0.026581,0.062896,-0.023883,0.000424,-0.001546,-0.010935,0.016667,0.005521,-0.000328,0.000120,0.011903
2011-04-30,0.048805,0.045559,0.027327,0.056258,0.015976,0.018414,0.046633,0.058671,0.019034,0.000045,0.028962,0.023780
2011-05-31,-0.013545,-0.051724,-0.029400,-0.022061,0.001333,0.025054,0.010133,-0.041565,0.000000,-0.000600,-0.011215,0.002659
2011-06-30,-0.000097,-0.042645,-0.009355,-0.012108,-0.005705,-0.005043,-0.031060,-0.042581,-0.010776,-0.000327,-0.016869,0.007548
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,0.000935,0.027596,0.021521,0.048009,0.013605,0.006165,0.018913,0.067995,0.016237,0.003133,0.026856,0.013703
2025-02-28,0.013991,0.001366,0.011470,0.029531,0.009702,0.027997,0.037861,-0.040107,-0.005952,0.002615,-0.012695,0.021572
2025-03-31,0.010353,0.022727,0.011340,0.001839,-0.010858,0.003420,-0.023382,-0.060826,-0.013237,0.002843,-0.055719,0.006797


In [8]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2011-02-28,0.007027,0.042120,-0.000027,0.035908,0.014763,-0.001674,0.045614,0.040556,0.002993,0.035147,0.007618
2011-03-31,0.008755,0.026909,0.063224,-0.023555,0.000752,-0.001218,-0.010607,0.016995,0.005849,0.000448,0.012231
2011-04-30,0.048760,0.045514,0.027283,0.056214,0.015932,0.018369,0.046589,0.058627,0.018989,0.028917,0.023735
2011-05-31,-0.012945,-0.051124,-0.028800,-0.021461,0.001933,0.025654,0.010733,-0.040965,0.000600,-0.010615,0.003259
2011-06-30,0.000230,-0.042318,-0.009027,-0.011781,-0.005378,-0.004716,-0.030733,-0.042254,-0.010449,-0.016542,0.007876
...,...,...,...,...,...,...,...,...,...,...,...
2025-01-31,-0.002198,0.024463,0.018388,0.044877,0.010472,0.003033,0.015780,0.064862,0.013104,0.023724,0.010570
2025-02-28,0.011376,-0.001250,0.008855,0.026915,0.007087,0.025382,0.035246,-0.042722,-0.008567,-0.015310,0.018957
2025-03-31,0.007510,0.019885,0.008497,-0.001004,-0.013701,0.000577,-0.026225,-0.063669,-0.016079,-0.058562,0.003954


In [9]:
with pd.ExcelWriter('../data/multi_asset_etf_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')